In [138]:
###IMPORTS###################################
import csv
import itertools

import StringIO
import requests

import json

In [5]:
def process_tweet(tweet):
    if tweet.find('"') != -1:
        tweet = tweet.replace('"', '""')
        tweet = '"' + tweet + '"'
    elif tweet.find(',') != -1:
        tweet = '"' + tweet + '"'
    
    return tweet

In [6]:
process_tweet("Just a ,comma")

'"Just a ,comma"'

In [7]:
process_tweet('Just a "double" quote')

'"Just a ""double"" quote"'

In [8]:
process_tweet('A comma, and a "double" quote')

'"A comma, and a ""double"" quote"'

In [33]:
def convert_training_csv_to_watson_csv_format(input_csv_name, group_id, output_csv_name): 
    # Converts an existing training csv file. The output file should
    # contain only the 11,000 lines of your group's specific training set.
    #
    # Inputs:
    #   input_csv - a string containing the name of the original csv file
    #		ex. "my_file.csv"
    #
    #	output_csv - a string containing the name of the output csv file
    #		ex. "my_output_file.csv"
    #
    # Returns:
    #	None
    
    class_zero_data = [group_id * 5500, (group_id + 1) * 5500]
    class_four_data = [group_id * 5500 + 800000, (group_id + 1) * 5500 + 800000]

    with open(input_csv_name, 'r') as _input:
        reader = csv.reader(_input)
        
        with open(output_csv_name, 'w') as _output:
            for row in itertools.islice(reader, *class_zero_data):
                tweet = process_tweet(row[5].strip())
                _output.write(tweet + "," + row[0] + '\n')
            for row in itertools.islice(reader, *class_four_data):
                tweet = process_tweet(row[5].strip())
                _output.write(tweet + "," + row[0] + '\n')

    return

In [34]:
convert_training_csv_to_watson_csv_format('datasets/training.1600000.processed.noemoticon.csv', 4, 'datasets/training_11000_watson_style.csv')

In [30]:
output_buf = StringIO.StringIO()

In [31]:
convert_training_csv_to_watson_csv_format('datasets/training.1600000.processed.noemoticon.csv', 4, output_buf)

In [32]:
print output_buf.getvalue()

@Natalia_Bella not much to buy now Woolworth closed down,0
Kill me please -.- ...Oh crap school tommorow,0
@chriskeating re the labour general secretary meeting with Labour PM's aide - I posted the very same on facebook. BBC gone downhill,0
Whole day of homework ahead of name,0
hamlet...romeo n juliet...radio:ACTIVE live at Wembley...McFly tour DVD's too money to me,0
"@charleypearson haha, lucky you. i just got told one!  loubee is not happy!",0
@SpecialEmily aw he says thank you! Yea its lush here got dress&amp;flipflops on but i broke my sunnies  gettin new ones @ the beach tho,0
"@Bruno108 Ohhh, so much? But I have?! Ohh, I feel terrible!",0
Inspiration is laughing at me,0
can't believe match was cancelled   is officially gutted. . .,0
Is sitting at home bored  And doesnt want to go back to school tomorrow,0
"Off to work  Yeah, ad set Sunday!",0
100th update. im at uni,0
has been doin graphics work for aaaaaaaaaaaages now  x,0
OH!  I'll email it to you,0
GLaDOS: one of the most eng

In [62]:
def extract_subset_from_csv_file(input_csv_file, n_lines_to_extract, output_file_prefix='ibmTrain'):
    # Extracts n_lines_to_extract lines from a given csv file and writes them to 
    # an outputfile named ibmTrain#.csv (where # is n_lines_to_extract).
    #
    # Inputs: 
    #	input_csv - a string containing the name of the original csv file from which
    #		a subset of lines will be extracted
    #		ex. "my_file.csv"
    #	
    #	n_lines_to_extract - the number of lines to extract from the csv_file, as an integer
    #		ex. 500
    #
    #	output_file_prefix - a prefix for the output csv file. If unspecified, output files 
    #		are named 'ibmTrain#.csv', where # is the input parameter n_lines_to_extract.
    #		The csv must be in the "watson" 2-column format.
    #		
    # Returns:
    #	None

    counters = [0, 0]
    
    with open(input_csv_file, 'r') as _input:
        with open(output_file_prefix + str(n_lines_to_extract) + ".csv", 'w') as _output:
            for line in _input:

                _class = int(line[-2])
                if counters[_class / 4] < n_lines_to_extract:
                    _output.write(line)
                    counters[_class / 4] += 1

    return

In [55]:
temp_buf = StringIO.StringIO()

In [56]:
extract_subset_from_csv_file('datasets/training_11000_watson_style.csv', 3, temp_buf)

['@Natalia_Bella not much to buy now Woolworth closed down,0\n']
['Kill me please -.- ...Oh crap school tommorow,0\n']
["@chriskeating re the labour general secretary meeting with Labour PM's aide - I posted the very same on facebook. BBC gone downhill,0\n"]
['Whole day of homework ahead of name,0\n']
["hamlet...romeo n juliet...radio:ACTIVE live at Wembley...McFly tour DVD's too money to me,0\n"]
['"@charleypearson haha, lucky you. i just got told one!  loubee is not happy!",0\n']
['@SpecialEmily aw he says thank you! Yea its lush here got dress&amp;flipflops on but i broke my sunnies  gettin new ones @ the beach tho,0\n']
['"@Bruno108 Ohhh, so much? But I have?! Ohh, I feel terrible!",0\n']
['Inspiration is laughing at me,0\n']
["can't believe match was cancelled   is officially gutted. . .,0\n"]
['Is sitting at home bored  And doesnt want to go back to school tomorrow,0\n']
['"Off to work  Yeah, ad set Sunday!",0\n']
['100th update. im at uni,0\n']
['has been doin graphics work for 

In [57]:
print temp_buf.getvalue()

@Natalia_Bella not much to buy now Woolworth closed down,0
Kill me please -.- ...Oh crap school tommorow,0
@chriskeating re the labour general secretary meeting with Labour PM's aide - I posted the very same on facebook. BBC gone downhill,0
@SpacECadET_Zei LOL   Morning,4
@andytlr Yeah   http://twitpic.com/3nisx,4
"Goooooood Morning! Leaving for New York in an hour or so. Cool, I'll be in New York by tonight.",4



In [190]:
def create_classifier(username, password, n, input_file_prefix='ibmTrain'):
    # Creates a classifier using the NLClassifier service specified with username and password.
    # Training_data for the classifier provided using an existing csv file named
    # ibmTrain#.csv, where # is the input parameter n.
    #
    # Inputs:
    # 	username - username for the NLClassifier to be used, as a string
    #
    # 	password - password for the NLClassifier to be used, as a string
    #
    #	n - identification number for the input_file, as an integer
    #		ex. 500
    #
    #	input_file_prefix - a prefix for the input csv file, as a string.
    #		If unspecified data will be collected from an existing csv file 
    #		named 'ibmTrain#.csv', where # is the input parameter n.
    #		The csv must be in the "watson" 2-column format.
    #
    # Returns:
    # 	A dictionary containing the response code of the classifier call, will all the fields 
    #	specified at
    #	http://www.ibm.com/smarterplanet/us/en/ibmwatson/developercloud/natural-language-classifier/api/v1/?curl#create_classifier
    #   
    #
    # Error Handling:
    #	This function should throw an exception if the create classifier call fails for any reason
    #	or if the input csv file does not exist or cannot be read.
    #
    
    url = "https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers"
    training_metadata = {'language': 'en', 'name': 'Classifier {}'.format(n)}
    data = {'training_metadata': json.dumps(training_metadata)}
    
    # Open will throw an IOError Exception if the input file doesn't exist or can't be read
    with open('{}{}.csv'.format(input_file_prefix, n), 'r') as input_file:
        files = [('training_data', input_file)]
        response = requests.post(url, auth=(username, password), data=data, files=files)
        
    if not response.ok:
        raise Exception("Error creating classifier")
    
    return response.json()

In [189]:
good.json()

{u'classifier_id': u'c7fa4ax22-nlc-1282',
 u'created': u'2016-02-15T02:59:55.601Z',
 u'language': u'en',
 u'name': u'Classifier 501',
 u'status': u'Training',
 u'status_description': u'The classifier instance is in its training phase, not yet ready to accept classify requests',
 u'url': u'https://gateway.watsonplatform.net/natural-language-classifier/api/v1/classifiers/c7fa4ax22-nlc-1282'}

In [177]:
bad.ok

False

##### Main

In [65]:
### STEP 1: Convert csv file into two-field watson format
input_csv_name = 'datasets/training.1600000.processed.noemoticon.csv'

#DO NOT CHANGE THE NAME OF THIS FILE
output_csv_name = 'datasets/training_11000_watson_style.csv'

convert_training_csv_to_watson_csv_format(input_csv_name, output_csv_name)

In [67]:
### STEP 2: Save 11 subsets in the new format into ibmTrain#.csv files

#TODO: extract all 11 subsets and write the 11 new ibmTrain#.csv files
#
# you should make use of the following function call:
#
for n_lines_to_extract in [500, 2500, 5000]:
    extract_subset_from_csv_file(output_csv_name, n_lines_to_extract, 'datasets/ibmTrain')

In [171]:
### STEP 3: Create the classifiers using Watson

#TODO: Create all 11 classifiers using the csv files of the subsets produced in STEP 2
# 
#
# you should make use of the following function call
username = "2bd0e6c7-5784-4967-860c-a9778754fdee"
password = "rFs4Solusscl"

for n in [500, 2500, 5000]:
    create_classifier(username, password, n, input_file_prefix='datasets/ibmTrain')